Aggregation in SQL
-----

In [2]:
%load_ext sql
%config SqlMagic.style = '_DEPRECATED_DEFAULT'
%sql sqlite:///world-db

**Exercise #1**: Write a query that outputs for each country the population of the most populated city, for countries with at least 10 cities.

In [ ]:
%%sql
SELECT Country.Name AS country, MAX(City.Population) AS pop
FROM Country
JOIN City ON Country.Code = City.CountryCode
GROUP BY Country.Name
HAVING COUNT(City.ID) > 10

In [ ]:
%%sql
SELECT C.NAME AS Country, MAX(T.Population) AS N
FROM City T, Country C
WHERE C.Code = T.CountryCode
GROUP BY C.Name
HAVING COUNT(T.ID) > 9
ORDER BY N DESC ;

Let's see how the use of `HAVING` compares with the use of correlated queries. Suppose that we want to find the names of the countries that have more than 10 cities with population at least 1 million. Here is a nested query that computes that: 

In [3]:
%sql SELECT * FROM City LIMIT 2;

 * sqlite:///world-db
Done.


ID,Name,CountryCode,District,Population
1,Kabul,AFG,Kabol,1780000
2,Qandahar,AFG,Qandahar,237500


In [4]:
%sql SELECT * FROM Country LIMIT 1;

 * sqlite:///world-db
Done.


Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
ABW,Aruba,North America,Caribbean,193.0,None,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129,AW


In [3]:
%%sql
SELECT C.name
FROM Country C
WHERE (SELECT COUNT(*) 
       FROM City
       WHERE City.CountryCode=C.Code
       AND City.Population >= 1000000) > 10; 

 * sqlite:///world-db
Done.


Name
Brazil
China
India
Japan
Russian Federation


We can measure the execution time of the query using `%time`.

In [4]:
%time %sql SELECT C.name FROM Country C WHERE (SELECT COUNT(*) FROM City WHERE City.CountryCode=C.Code AND City.Population >= 1000000) > 10; 

 * sqlite:///world-db
Done.
CPU times: user 34.6 ms, sys: 957 µs, total: 35.6 ms
Wall time: 35.3 ms


**Exercise #2**: Write the above query using `HAVING` and time its execution. How much faster does it run?

In [6]:
%time %sql SELECT C.name FROM Country C, City T WHERE T.CountryCode=C.Code AND T.Population >= 1000000 GROUP BY C.name HAVING COUNT(*)> 10; 

 * sqlite:///world-db
Done.
CPU times: user 2.98 ms, sys: 1.39 ms, total: 4.37 ms
Wall time: 4.14 ms
